In [ ]:
import pandas as pd

# Sort bookings

Detta sorterar från alla bokningar osorterade.

In [ ]:

year = 2024
days_before = 56

filenames = [f"alla_bokningar_{year}.csv"]
sammanfogad_df = pd.DataFrame()

for file_name in filenames:
    df = pd.read_csv(file_name)
    df['DateFrom'] = pd.to_datetime(df['DateFrom'], format='%Y-%m-%d %H:%M:%S.%f', errors="coerce")
    df['DateTo'] = pd.to_datetime(df['DateTo'], format='%Y-%m-%d %H:%M:%S.%f', errors="coerce")
    df = df.dropna(subset=['DateFrom', 'DateTo'])
    nat_counts = df.isna().sum()

    df = df.dropna(subset=['DateFrom', 'DateTo'])
    sammanfogad_df = pd.concat([sammanfogad_df, df], ignore_index=True)
df_sorted = sammanfogad_df.sort_values(by='DateFrom', ascending=True)
df_sorted['DateFrom'] = pd.to_datetime(df_sorted['DateFrom'])
df_sorted['DateTo'] = pd.to_datetime(df_sorted['DateTo'])

def generate_date_range(row):
    return pd.date_range(start=row['DateFrom'], end=row['DateTo']).tolist()

df_sorted['DateRange'] = df.apply(generate_date_range, axis=1)
df_sorted = df_sorted.dropna(subset=['DateFrom', 'DateTo','DateRange'])

df_sorted['OrderDate'] = pd.to_datetime(df_sorted['OrderDate'])

rows = []
for _, row in df_sorted.iterrows():
    for visit_date in row['DateRange']:
        days_in_advance = (visit_date - row['OrderDate']).days
        if days_in_advance >= days_before:
            rows.append({'date': visit_date, 'DaysInAdvance': days_in_advance})

expanded_bookings_df = pd.DataFrame(rows)

count_bookings_df = expanded_bookings_df.groupby('date').size().reset_index(name='BookingCount')
print(count_bookings_df)
count_bookings_df.to_csv(f'amount_bookings_{days_before}_days_before_sorted_on_dates_{year}.csv')


# Combine to one file

In [ ]:
days_before = 56
files = [f"amount_bookings_{days_before}_days_before_sorted_on_dates_2021.csv",f"amount_bookings_{days_before}_days_before_sorted_on_dates_2023.csv",
         f"amount_bookings_{days_before}_days_before_sorted_on_dates_2022.csv",f"amount_bookings_{days_before}_days_before_sorted_on_dates_2024.csv"]

dfs = [pd.read_csv(file) for file in files]
combined_df = pd.concat(dfs)
sorted_comb_df = combined_df.sort_values('VisitDate')

print(sorted_comb_df)
sorted_comb_df.to_csv(f"amount_of_bookings_{days_before}_days_before_all_years.csv")

# Merge Visitor Data with rain data

In [ ]:
original_df = pd.read_csv('visitor_data_with_weather.csv')
original_df = original_df[['date','visitors']]
original_df = original_df[original_df['date'].between('2020-01-01', '2024-12-31')].copy()
original_df['date'] = pd.to_datetime(original_df['date'])

df_with_rain = pd.read_csv('smhi_rain.csv', sep=';', index_col=False, low_memory=False)
df_with_rain['date'] = pd.to_datetime(df['date'], errors='coerce')
df_with_rain_sorted = df_with_rain.sort_values(by=['date', 'Tid'])
df_with_rain_sorted['Nederbördsmängd'] = pd.to_numeric(df_with_rain_sorted['Nederbördsmängd'], errors='coerce')
df_with_rain_sorted['Tid'] = pd.to_datetime(df_with_rain_sorted['Tid'], format='%H:%M:%S').dt.time

df_with_rain_sorted['weather_rain'] = ((df_sorted['Tid'] >= pd.to_datetime('07:00:00').time()) & 
                     (df_sorted['Tid'] <= pd.to_datetime('18:00:00').time()) & 
                     (df_sorted['Nederbördsmängd'] > 0.0)).astype(int)

df_daily_rain = df_with_rain_sorted.groupby('date')['weather_rain'].max().reset_index()

In [ ]:
rain_merge = original_df.merge(df_daily_rain[['date','weather_rain']], on='date',how='left')
print(rain_merge)

# Merge with temperature

In [ ]:
df_temp = pd.read_csv('smhi_temp_max.csv', sep=';', index_col=False)
df_temp['date'] = pd.to_datetime(df_temp['date'], errors='coerce')
df_temp = df_temp.sort_values(by=['date', 'Tid'])
df_temp['Tid'] = pd.to_datetime(df_temp['Tid'], format='%H:%M:%S').dt.time
df_filtered = df_temp[df_temp['Tid'] == pd.to_datetime('18:00:00').time()]
df_filtered = df_filtered.rename(columns={'Lufttemperatur':'weather_temp'})

all_weather_merged = rain_merge.merge(df_filtered[['date','weather_temp']],on='date',how='left')
print(all_weather_merged)

# Merge with bookings

In [ ]:
file_days = list(range(1, 11)) + [14, 21, 35, 56]

for day in file_days:
    df_bookings = pd.read_csv(f'amount_of_bookings_{day}_days_before_all_years.csv')
    df_bookings['date'] = pd.to_datetime(df_bookings['VisitDate'])
    column_name = f'BookingCount_{day}'
    df_bookings.rename(columns={'BookingCount': column_name}, inplace=True)
    all_weather_merged = all_weather_merged.merge(df_bookings[['date', column_name]], on='date', how='left')

print('Final columns in all_weather_merged:', all_weather_merged.columns)

print(all_weather_merged)
